# Projeto Final da Disciplina de Grafos - UFAL 2022.2
By: Lucas Barbosa Leite Silva

## Conhecendo do Problema
Utilizaremos um dataset provido pela Agência Nacional de Aviação Civil (ANAC). Este data set está disponível gratuitamente no site da ANAC (https://www.anac.gov.br/acesso-a-informacao/dados-abertos/areas-de-atuacao/voos-e-operacoes-aereas/dados-estatisticos-do-transporte-aereo) e contém registros de voos que ocorreram no Brasil desde do ano 2000. <br>
A ideia original é:
- Montar um grafo contendo as rotas;
- Cada rota terá um conjunto de pesos: custo em reais da rota para a empresa por passageiro, tempo para realizar aquela rota e quantidade de conexões;
- Gerar uma ávore geradora mínima das rotas, a fim de conectar todos os vértices com o menor peso possível;
- Utilizar a busca em profundidade para ir de um ponto a outro do grafo com o menor custo (heurístico);

## Preparação dos Dados

### Importando planilha

In [ ]:
pip install pandas

In [61]:
import pandas as pd

In [62]:
planilha = pd.read_csv(
    'Dados_Estatisticos.csv', 
    sep=';'
  )

### Visualizando os Dados

In [63]:
planilha.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 966166 entries, 0 to 966165
Data columns (total 38 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   EMPRESA_SIGLA                    966166 non-null  object 
 1   EMPRESA_NOME                     966166 non-null  object 
 2   EMPRESA_NACIONALIDADE            966166 non-null  object 
 3   ANO                              966166 non-null  int64  
 4   MES                              966166 non-null  int64  
 5   AEROPORTO_DE_ORIGEM_SIGLA        960952 non-null  object 
 6   AEROPORTO_DE_ORIGEM_NOME         960952 non-null  object 
 7   AEROPORTO_DE_ORIGEM_UF           827939 non-null  object 
 8   AEROPORTO_DE_ORIGEM_REGIAO       827939 non-null  object 
 9   AEROPORTO_DE_ORIGEM_PAIS         960952 non-null  object 
 10  AEROPORTO_DE_ORIGEM_CONTINENTE   960952 non-null  object 
 11  AEROPORTO_DE_DESTINO_SIGLA       966166 non-null  object 
 12  AE

In [64]:
planilha.head()

,EMPRESA_SIGLA,EMPRESA_NOME,EMPRESA_NACIONALIDADE,ANO,MES,AEROPORTO_DE_ORIGEM_SIGLA,AEROPORTO_DE_ORIGEM_NOME,AEROPORTO_DE_ORIGEM_UF,AEROPORTO_DE_ORIGEM_REGIAO,AEROPORTO_DE_ORIGEM_PAIS,...,COMBUSTIVEL_LITROS,DISTANCIA_VOADA_KM,DECOLAGENS,CARGA_PAGA_KM,CARGA_GRATIS_KM,CORREIO_KM,ASSENTOS,PAYLOAD,HORAS_VOADAS,BAGAGEM_KG
0,AAL,"AMERICAN AIRLINES, INC.",ESTRANGEIRA,2000,1,KDFW,"DALLAS & FORT WORTH, TEXAS",NaN,NaN,ESTADOS UNIDOS DA AMÉRICA,...,NaN,247320.0,30.0,8.068900e+08,0.0,601812.0,6330.0,1050000.0,"409,08",NaN
1,AAL,"AMERICAN AIRLINES, INC.",ESTRANGEIRA,2000,1,KJFK,"NEW YORK, NEW YORK",NaN,NaN,ESTADOS UNIDOS DA AMÉRICA,...,NaN,224141.0,29.0,3.715020e+08,0.0,62094800.0,6119.0,464000.0,"418,1",NaN
2,AAL,"AMERICAN AIRLINES, INC.",ESTRANGEIRA,2000,1,KJFK,"NEW YORK, NEW YORK",NaN,NaN,ESTADOS UNIDOS DA AMÉRICA,...,NaN,222256.0,29.0,1.494460e+09,0.0,15174700.0,6119.0,1015000.0,"411,5",NaN
3,AAL,"AMERICAN AIRLINES, INC.",ESTRANGEIRA,2000,1,KMIA,"MIAMI, FLORIDA",NaN,NaN,ESTADOS UNIDOS DA AMÉRICA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AAL,"AMERICAN AIRLINES, INC.",ESTRANGEIRA,2000,1,KMIA,"MIAMI, FLORIDA",NaN,NaN,ESTADOS UNIDOS DA AMÉRICA,...,NaN,208227.0,31.0,8.119980e+08,46495100.0,230689000.0,6541.0,1077000.0,"491,8",NaN


### Aplicando Filtros

In [65]:
planilha = planilha[
    (planilha["AEROPORTO_DE_ORIGEM_PAIS"]=="BRASIL") & 
    (planilha["AEROPORTO_DE_DESTINO_PAIS"]=="BRASIL") &
    (planilha["ANO"] == 2023)
]

In [66]:
planilha.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7318 entries, 955925 to 966043
Data columns (total 38 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   EMPRESA_SIGLA                    7318 non-null   object 
 1   EMPRESA_NOME                     7318 non-null   object 
 2   EMPRESA_NACIONALIDADE            7318 non-null   object 
 3   ANO                              7318 non-null   int64  
 4   MES                              7318 non-null   int64  
 5   AEROPORTO_DE_ORIGEM_SIGLA        7318 non-null   object 
 6   AEROPORTO_DE_ORIGEM_NOME         7318 non-null   object 
 7   AEROPORTO_DE_ORIGEM_UF           7318 non-null   object 
 8   AEROPORTO_DE_ORIGEM_REGIAO       7318 non-null   object 
 9   AEROPORTO_DE_ORIGEM_PAIS         7318 non-null   object 
 10  AEROPORTO_DE_ORIGEM_CONTINENTE   7318 non-null   object 
 11  AEROPORTO_DE_DESTINO_SIGLA       7318 non-null   object 
 12  AEROPORTO_DE_DESTI

### Selecionando colunas

In [67]:
planilha = planilha[['ANO','AEROPORTO_DE_ORIGEM_SIGLA','AEROPORTO_DE_ORIGEM_NOME','AEROPORTO_DE_ORIGEM_UF','AEROPORTO_DE_DESTINO_SIGLA','AEROPORTO_DE_DESTINO_NOME', 'AEROPORTO_DE_DESTINO_UF','COMBUSTIVEL_LITROS', 'DISTANCIA_VOADA_KM', 'HORAS_VOADAS', 'DECOLAGENS','PASSAGEIROS_PAGOS']]

In [68]:
display(planilha)

,ANO,AEROPORTO_DE_ORIGEM_SIGLA,AEROPORTO_DE_ORIGEM_NOME,AEROPORTO_DE_ORIGEM_UF,AEROPORTO_DE_DESTINO_SIGLA,AEROPORTO_DE_DESTINO_NOME,AEROPORTO_DE_DESTINO_UF,COMBUSTIVEL_LITROS,DISTANCIA_VOADA_KM,HORAS_VOADAS,DECOLAGENS,PASSAGEIROS_PAGOS
955925,2023,SBSV,SALVADOR,BA,SIRI,MARAÚ,BA,1160.0,1290.0,"5,285",10.0,29.0
955926,2023,SBSV,SALVADOR,BA,SNCL,CAIRU,BA,4736.0,5312.0,"24,4",64.0,253.0
955927,2023,SIRI,MARAÚ,BA,SBSV,SALVADOR,BA,1160.0,1290.0,"5,299",10.0,54.0
955928,2023,SNCL,CAIRU,BA,SBSV,SALVADOR,BA,4736.0,5312.0,"25,268",64.0,457.0
955929,2023,SBSV,SALVADOR,BA,SBSV,SALVADOR,BA,74.0,NaN,"1,2",1.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...
966039,2023,SBGR,GUARULHOS,SP,SBVT,VITÓRIA,ES,NaN,NaN,NaN,NaN,0.0
966040,2023,SBPA,PORTO ALEGRE,RS,SBGR,GUARULHOS,SP,192510.0,19918.0,"39,1333",23.0,0.0
966041,2023,SBRF,RECIFE,PE,SBGL,RIO DE JANEIRO,RJ,7517.0,1859.0,"2,88333",1.0,0.0
966042,2023,SBVT,VITÓRIA,ES,SBGR,GUARULHOS,SP,4134.0,730.0,"1,66667",1.0,0.0


### Removendo ruídos

In [69]:
planilha.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7318 entries, 955925 to 966043
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   ANO                         7318 non-null   int64  
 1   AEROPORTO_DE_ORIGEM_SIGLA   7318 non-null   object 
 2   AEROPORTO_DE_ORIGEM_NOME    7318 non-null   object 
 3   AEROPORTO_DE_ORIGEM_UF      7318 non-null   object 
 4   AEROPORTO_DE_DESTINO_SIGLA  7318 non-null   object 
 5   AEROPORTO_DE_DESTINO_NOME   7318 non-null   object 
 6   AEROPORTO_DE_DESTINO_UF     7318 non-null   object 
 7   COMBUSTIVEL_LITROS          7025 non-null   float64
 8   DISTANCIA_VOADA_KM          7023 non-null   float64
 9   HORAS_VOADAS                7025 non-null   object 
 10  DECOLAGENS                  7025 non-null   float64
 11  PASSAGEIROS_PAGOS           7283 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 743.2+ KB


In [70]:
planilha = planilha.dropna(how="any", axis=0)

In [71]:
planilha['DECOLAGENS'] = pd.to_numeric(planilha["DECOLAGENS"], downcast='integer')

In [72]:
planilha['PASSAGEIROS_PAGOS'] = pd.to_numeric(planilha["PASSAGEIROS_PAGOS"], downcast='integer')

In [73]:
planilha['HORAS_VOADAS'] = planilha['HORAS_VOADAS'].str.replace(",", ".").astype(float)

In [75]:
remove = planilha.loc[
    (planilha['PASSAGEIROS_PAGOS'] == 0.0)
]
planilha = planilha.drop(remove.index)

In [76]:
display(planilha)
planilha.info()

,ANO,AEROPORTO_DE_ORIGEM_SIGLA,AEROPORTO_DE_ORIGEM_NOME,AEROPORTO_DE_ORIGEM_UF,AEROPORTO_DE_DESTINO_SIGLA,AEROPORTO_DE_DESTINO_NOME,AEROPORTO_DE_DESTINO_UF,COMBUSTIVEL_LITROS,DISTANCIA_VOADA_KM,HORAS_VOADAS,DECOLAGENS,PASSAGEIROS_PAGOS
955925,2023,SBSV,SALVADOR,BA,SIRI,MARAÚ,BA,1160.0,1290.0,5.2850,10,29
955926,2023,SBSV,SALVADOR,BA,SNCL,CAIRU,BA,4736.0,5312.0,24.4000,64,253
955927,2023,SIRI,MARAÚ,BA,SBSV,SALVADOR,BA,1160.0,1290.0,5.2990,10,54
955928,2023,SNCL,CAIRU,BA,SBSV,SALVADOR,BA,4736.0,5312.0,25.2680,64,457
955930,2023,SBSV,SALVADOR,BA,SIRI,MARAÚ,BA,760.0,1032.0,3.6990,8,39
...,...,...,...,...,...,...,...,...,...,...,...,...
965723,2023,SBVT,VITÓRIA,ES,SBRJ,RIO DE JANEIRO,RJ,346761.0,54051.0,137.2330,129,11263
965724,2023,SBVT,VITÓRIA,ES,SBSP,SÃO PAULO,SP,29419.0,5292.0,11.4833,7,634
965725,2023,SBVT,VITÓRIA,ES,SBSP,SÃO PAULO,SP,804521.0,138348.0,291.6830,183,18873
965727,2023,SBZM,GOIANÁ,MG,SBGR,GUARULHOS,SP,3289.0,401.0,1.3500,1,85


<class 'pandas.core.frame.DataFrame'>
Index: 5270 entries, 955925 to 965728
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   ANO                         5270 non-null   int64  
 1   AEROPORTO_DE_ORIGEM_SIGLA   5270 non-null   object 
 2   AEROPORTO_DE_ORIGEM_NOME    5270 non-null   object 
 3   AEROPORTO_DE_ORIGEM_UF      5270 non-null   object 
 4   AEROPORTO_DE_DESTINO_SIGLA  5270 non-null   object 
 5   AEROPORTO_DE_DESTINO_NOME   5270 non-null   object 
 6   AEROPORTO_DE_DESTINO_UF     5270 non-null   object 
 7   COMBUSTIVEL_LITROS          5270 non-null   float64
 8   DISTANCIA_VOADA_KM          5270 non-null   float64
 9   HORAS_VOADAS                5270 non-null   float64
 10  DECOLAGENS                  5270 non-null   int16  
 11  PASSAGEIROS_PAGOS           5270 non-null   int32  
dtypes: float64(3), int16(1), int32(1), int64(1), object(6)
memory usage: 483.8+ KB


## Modelando Problema

#### Construir meu grafo:
- Usando dicionário
- Chave: coluna AEROPORTO_DE_ORIGEM_SIGLA - AEROPORTO_DE_ORIGEM_UF
- E os valores serão um dicionário contendo o par AEROPORTO_DE_DESTINO_SIGLA - AEROPORTO_DE_DESTINO_NOME.
    - O valor do dicionário vai ser um outro dicionário onde as chaves vão ser DECOLAGENS e o valor será o peso da aresta: uma tupla contendo o preço em reais e o tempo de voo.

##### Calcular o custo da seguinte forma:
- **Peso:** (Custo de tempo + custo de combustivel) / Passageiros Pagos
- **Custo de tempo:** Horas voadas * (precoHoraPiloto + precoHoraCoPiloto + precoHoraComissario * 3 + Custo de manutenção por hora)
- **Custo de combustivel:** Combustível * preço do combustível
- Arrendondar o preço da melhor forma

##### Definindo custos
- **Piloto:** Salario: 18000; Horas de voo mesais: 85. Preço por hora: ~ R$212,00
- **Copiloto:** Salario: 16000; Horas de voo mesais: 85. Preço por hora: ~ R$ 189,00
- **Comissário de bordo:** Salário 4000, horas mesais trabalhadas 80). Preço por hora: ~ R$ 50,00
- Custo de manutenção da aeronave: Preço por hora: R$ 150,00
- Custo do combustível: 5,3 por  litro

Exemplo do grafo:<br>
{<br>
&ensp;&ensp;"SBSV - BA": {<br>
&ensp;&ensp;&ensp;&ensp;"SIRI - BA":{<br>
&ensp;&ensp;&ensp;&ensp;&ensp;"10": (279, 5,6)<br>
&ensp;&ensp;&ensp;&ensp;},<br>
&ensp;&ensp;&ensp;&ensp;"SNCL - BA":{<br>
&ensp;&ensp;&ensp;&ensp;&ensp;"64": 135<br>
&ensp;&ensp;&ensp;&ensp;}<br>
&ensp;&ensp;"SIRI - BA":{<br>
&ensp;&ensp;&ensp;&ensp;"SBSV - BA":{<br>
&ensp;&ensp;&ensp;&ensp;&ensp;"10": 150<br>
&ensp;&ensp;&ensp;&ensp;}<br>
&ensp;&ensp;}<br>
}<br>


In [77]:
# variaveis de custo
horaPiloto = 212.0
horaCopiloto = 189.0
horaComissario = 50.0
horaManutencaoAeronave = 150.0
custoLitroCombustivel = 5.3

In [78]:
class Grafo:
    grafo = None
    quantidadeArestas = 0
    quantidadeVertices = 0
    aeroportos = None
    def __init__(self):
        self.grafo = {}
        self.aeroportos = {}

    def adicionarRota(self, origem, nomeOrigem, destino, nomeDestino, label, peso, tempo):
        #existe origem?
        if(origem in self.grafo):
        #sim
            #existe destino?
            if(destino in self.grafo[origem]):
            #sim
                #existe label?
                if(label in self.grafo[origem][destino]):
                    #sim
                    #fazer a média
                    novoPeso = round(peso + self.grafo[origem][destino][label][0]/2)
                    novoTempo = round(tempo + self.grafo[origem][destino][label][1]/2)
                    self.grafo[origem][destino][label] = (novoPeso, novoTempo)
                    self.quantidadeArestas+=1
                #nao
                else:
                    #adicionar a label
                    self.grafo[origem][destino][label] = (peso, tempo)
                    self.quantidadeArestas+=1
            #nao
            else:
                #adicionar "destino" : {"label" : peso}
                self.grafo[origem][destino] = {label : (peso, tempo)}
                self.quantidadeArestas+=1
        #nao
        else:
            #adicionar [origem] = {"destino" : {"label" : peso}}
            self.grafo[origem] = {destino : {label : (peso, tempo)}}
            self.aeroportos[origem] = nomeOrigem
            self.quantidadeArestas+=1
            self.quantidadeVertices+=1
        if(not(destino in self.grafo)):
            self.grafo[destino] = {}
            self.aeroportos[destino] = nomeDestino
            self.quantidadeVertices+=1

    
    def __str__(self):
        string = ""
        for origem in self.grafo.keys():
            string += origem+":"+"\n"
            for destino in self.grafo[origem].keys():
                string += "     "+destino+":"+"\n"
                for label, peso in self.grafo[origem][destino].items():
                    string +="         "+str(label)+": "+str(peso)+"\n"
        return string


In [90]:
grafo = Grafo()

In [91]:
for index, row in planilha.iterrows():
    AEROPORTO_DE_ORIGEM_SIGLA = row["AEROPORTO_DE_ORIGEM_SIGLA"]
    AEROPORTO_DE_ORIGEM_NOME = row["AEROPORTO_DE_ORIGEM_NOME"]
    AEROPORTO_DE_DESTINO_SIGLA = row["AEROPORTO_DE_DESTINO_SIGLA"]
    AEROPORTO_DE_DESTINO_NOME = row["AEROPORTO_DE_DESTINO_NOME"]
    AEROPORTO_DE_ORIGEM_UF = row["AEROPORTO_DE_ORIGEM_UF"]
    AEROPORTO_DE_DESTINO_UF = row["AEROPORTO_DE_DESTINO_UF"]
    COMBUSTIVEL_LITROS = row["COMBUSTIVEL_LITROS"]
    DISTANCIA_VOADA_KM = row["DISTANCIA_VOADA_KM"]
    HORAS_VOADAS = row["HORAS_VOADAS"]
    DECOLAGENS = row["DECOLAGENS"]
    PASSAGEIROS_PAGOS = row["PASSAGEIROS_PAGOS"]

    origem = AEROPORTO_DE_ORIGEM_SIGLA + " - " +AEROPORTO_DE_ORIGEM_UF
    destino = AEROPORTO_DE_DESTINO_SIGLA + " - " + AEROPORTO_DE_DESTINO_UF
    label = DECOLAGENS
    custoHoras = HORAS_VOADAS*(horaPiloto + horaCopiloto + (horaComissario*3) + horaManutencaoAeronave)
    custoCombustivel = COMBUSTIVEL_LITROS * custoLitroCombustivel
    peso = round( (custoHoras + custoCombustivel) / PASSAGEIROS_PAGOS)

    grafo.adicionarRota(origem=origem, nomeOrigem=AEROPORTO_DE_ORIGEM_NOME, destino=destino, nomeDestino=AEROPORTO_DE_DESTINO_NOME, label=label, peso=peso, tempo=HORAS_VOADAS)

In [92]:
print(str(grafo))

SBSV - BA:
     SIRI - BA:
         10: (340, 5.285)
         8: (170, 3.699)
         7: (362, 3.666)
     SNCL - BA:
         64: (167, 24.4)
         52: (127, 20.549)
         31: (112, 11.301)
     SBBR - DF:
         1: (251, 1.73333)
         90: (165, 174.183)
         3: (168, 5.75)
         73: (169, 141.433)
         86: (168, 169.233)
         4: (177, 7.35)
         58: (188, 107.133)
         56: (183, 104.567)
         2: (194, 3.75)
         63: (211, 118.567)
     SBCF - MG:
         20: (217, 37.0667)
         111: (191, 189.183)
         9: (260, 16.5833)
         102: (179, 169.033)
         3: (254, 6.66667)
         110: (196, 184.117)
         57: (167, 101.767)
         4: (173, 6.85)
         46: (155, 78.9)
         40: (175, 71.95)
         1: (321, 2.78333)
     SBCY - MT:
         4: (331, 11.2833)
         3: (293, 8.55)
         5: (297, 14.5333)
         1: (290, 2.83333)
     SBFI - PR:
         14: (485, 45.8667)
         7: (340, 22.75)
         1: (3

In [93]:
len(grafo.grafo)

164

In [94]:
print("Vértices: ", grafo.quantidadeVertices)
print("Arestas: ",grafo.quantidadeArestas)


Vértices:  164
Arestas:  5270


## Conexão com o banco de dados
Para a conexão com o banco de dados, vamos utilizar o Neo4j.<br>
- Baixar a imagem oficial no Docker
- Configurar as variáveis:
    - portas: 0.0.0.0:7473, 0.0.0.0:7474, 0.0.0.0:7687
    - volumes: /data e /logs
    - vaiáveis de ambiente: NEO4J_AUTH: neo4j/test1234

### Classe de Conexão com o Neo4j

In [ ]:
pip install neo4j

In [85]:
from neo4j import GraphDatabase

class ConectionNeo4j:

    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self.driver.close()

    #GERENCIAMENTO DE AEROPORTOS ---------------------------------------------------------------------
    def createAirport(self, sigla, nome):
        with self.driver.session() as session:
            greeting = session.execute_write(self.createAirportTX, sigla, nome)
            print(greeting)

    def deleteAirport(self, sigla):
        with self.driver.session() as session:
            greeting = session.execute_write(self.deleteAirportTX, sigla)
            print(greeting)

    @staticmethod
    def deleteAirportTX(tx, sigla):
        result = tx.run("MATCH (o:Airport {sigla: $sigla})"
                        "DETACH DELETE o", 
                        sigla=sigla
                        )
        return "Aeroporto deletado"

    @staticmethod
    def createAirportTX(tx, sigla, nome):
        result = tx.run("CREATE (o:Airport {sigla: $sigla, nome: $nome})", 
                        sigla=sigla,
                        nome=nome
                        )
        return "Aeroporto Criado"

    #GERENCIAMENTO DE ROTAS ROUTES ---------------------------------------------------------------------
    def createRote(self, origem, destino, label, peso, tempo):
        with self.driver.session() as session:
            greeting = session.execute_write(self.createRoteTX, origem, destino, label, peso, tempo)
            print(greeting)

    @staticmethod
    def createRoteTX(tx, origem, destino, label, peso, tempo):
        result = tx.run("MATCH (o:Airport {sigla: $origem}) "
                        "MATCH (d:Airport {sigla: $destino}) "
                        "CREATE (o)-[r:Route]->(d)"
                        "SET r.label = $label "
                        "SET r.peso = $peso "
                        "SET r.tempo = $tempo ", 
                        origem=origem,
                        destino=destino,
                        label=label,
                        peso=peso,
                        tempo=tempo
                        )
        return "Rota Padrão Criada"
    
    def deleteAllRoutes(self):
        with self.driver.session() as session:
            greeting = session.execute_write(self.deleteAllRoutesTX)
            print(greeting)

    @staticmethod
    def deleteAllRoutesTX(tx):
        result = tx.run("MATCH ()-[r:Route]->()"
                        "DETACH DELETE r")
        return "Rotas Padrão Deletadas"
    
    #GERENCIAMENTO DE ROTAS PESO ---------------------------------------------------------------------
    def createRotePeso(self, origem, destino, peso, tempo, label):
        with self.driver.session() as session:
            greeting = session.execute_write(self.createRotePesoTX, origem, destino, peso, tempo, label)
            print(greeting)

    @staticmethod
    def createRotePesoTX(tx, origem, destino, label, peso, tempo):
        result = tx.run("MATCH (o:Airport {sigla: $origem}) "
                        "MATCH (d:Airport {sigla: $destino}) "
                        "CREATE (o)-[r:RoutePeso]->(d)"
                        "SET r.peso = $peso "
                        "SET r.tempo = $tempo "
                        "SET r.label = $label ",
                        origem=origem,
                        destino=destino,
                        label=label,
                        peso=peso,
                        tempo=tempo
                        )
        return "Rota Peso Criada"
    
    def deleteAllRoutesPeso(self):
        with self.driver.session() as session:
            greeting = session.execute_write(self.deleteAllRoutesPesoTX)
            print(greeting)

    @staticmethod
    def deleteAllRoutesPesoTX(tx):
        result = tx.run("MATCH ()-[r:RoutePeso]->()"
                        "DETACH DELETE r")
        return "Rotas Peso Deletadas"
    
    #GERENCIAMENTO DE ROTAS PRIM ---------------------------------------------------------------------
    def createRotePrim(self, origem, destino, peso, tempo, label):
        with self.driver.session() as session:
            greeting = session.execute_write(self.createRotePrimTX, origem, destino, peso, tempo, label)
            print(greeting)

    @staticmethod
    def createRotePrimTX(tx, origem, destino, label, peso, tempo):
        result = tx.run("MATCH (o:Airport {sigla: $origem}) "
                        "MATCH (d:Airport {sigla: $destino}) "
                        "CREATE (o)-[r:RoutePrim]->(d)"
                        "SET r.peso = $peso "
                        "SET r.tempo = $tempo "
                        "SET r.label = $label ",
                        origem=origem,
                        destino=destino,
                        label=label,
                        peso=peso,
                        tempo=tempo
                        )
        return "Rota Prim Criada"
    
    def deleteAllRoutesPrim(self):
        with self.driver.session() as session:
            greeting = session.execute_write(self.deleteAllRoutesPrimTX)
            print(greeting)

    @staticmethod
    def deleteAllRoutesPrimTX(tx):
        result = tx.run("MATCH ()-[r:RoutePrim]->()"
                        "DETACH DELETE r")
        return "Rotas Prim Deletadas"
    
    #GERENCIAMENTO DE ROTAS BUSCA ---------------------------------------------------------------------
    def createRoteBusca(self, origem, destino, peso, tempo, label):
        with self.driver.session() as session:
            greeting = session.execute_write(self.createRoteBuscaTX, origem, destino, peso, tempo, label)
            print(greeting)

    @staticmethod
    def createRoteBuscaTX(tx, origem, destino, label, peso, tempo):
        result = tx.run("MATCH (o:Airport {sigla: $origem}) "
                        "MATCH (d:Airport {sigla: $destino}) "
                        "CREATE (o)-[r:Busca]->(d)"
                        "SET r.peso = $peso "
                        "SET r.tempo = $tempo "
                        "SET r.label = $label ",
                        origem=origem,
                        destino=destino,
                        label=label,
                        peso=peso,
                        tempo=tempo
                        )
        return "Rota Busca Criada"
    
    def deleteAllRoutesBusca(self):
        with self.driver.session() as session:
            greeting = session.execute_write(self.deleteAllRoutesBuscaTX)
            print(greeting)

    @staticmethod
    def deleteAllRoutesBuscaTX(tx):
        result = tx.run("MATCH ()-[r:Busca]->()"
                        "DETACH DELETE r")
        return "Rotas Busca Deletadas"

### Funções úteis

In [95]:
def criarAeroportos(grafo):
    conn = ConectionNeo4j("bolt://localhost:7687", "neo4j", "password")
    for x in grafo.grafo.keys():
        conn.createAirport(sigla=x, nome=grafo.aeroportos[x])
    conn.close()

def criarRotasRoutes(grafo):
    conn = ConectionNeo4j("bolt://localhost:7687", "neo4j", "password")
    for origem in grafo.grafo.keys():
        for destino in grafo.grafo[origem].keys():
            for label, value in grafo.grafo[origem][destino].items():
                conn.createRote(origem=origem,destino=destino, label=label, peso=value[0], tempo=value[1])
    conn.close()

def criarRotasPeso(grafo):
    conn = ConectionNeo4j("bolt://localhost:7687", "neo4j", "password")
    for origem in grafo.grafo.keys():
        for destino, value in grafo.grafo[origem].items():
            peso, tempo, label = value
            conn.createRotePeso(origem=origem,destino=destino, label=label, peso=peso, tempo=tempo)
    conn.close()

def criarRotasPrim(grafo):
    conn = ConectionNeo4j("bolt://localhost:7687", "neo4j", "password")
    for origem in grafo.grafo.keys():
        for destino, value in grafo.grafo[origem].items():
            peso, tempo, label = value
            conn.createRotePrim(origem=origem,destino=destino, label=label, peso=peso, tempo=tempo)
    conn.close()

def criarRotasBusca(grafo, rota):
    conn = ConectionNeo4j("bolt://localhost:7687", "neo4j", "password")
    for i in range(len(rota)-1):    
        origem = rota[i]
        destino = rota[i+1]
        peso, tempo, label = grafo.grafo[origem][destino]
        conn.createRoteBusca(origem=origem,destino=destino, label=label, peso=peso, tempo=tempo)
    conn.close()

def deletarAeroportos(grafo):
    conn = ConectionNeo4j("bolt://localhost:7687", "neo4j", "password")
    for x in grafo.grafo.keys():
        conn.deleteAirport(sigla=x)
    conn.close()    

def deletarAllRoutesRoutes():
    conn = ConectionNeo4j("bolt://localhost:7687", "neo4j", "password")
    conn.deleteAllRoutes()
    conn.close()

def deletarAllRoutesPeso():
    conn = ConectionNeo4j("bolt://localhost:7687", "neo4j", "password")
    conn.deleteAllRoutesPeso()
    conn.close()

def deletarAllRoutesPrim():
    conn = ConectionNeo4j("bolt://localhost:7687", "neo4j", "password")
    conn.deleteAllRoutesPrim()
    conn.close()

def deletarAllRoutesBusca():
    conn = ConectionNeo4j("bolt://localhost:7687", "neo4j", "password")
    conn.deleteAllRoutesBusca()
    conn.close()


In [96]:
criarAeroportos(grafo)
criarRotasRoutes(grafo)

Aeroporto Criado
Aeroporto Criado
Aeroporto Criado
Aeroporto Criado
Aeroporto Criado
Aeroporto Criado
Aeroporto Criado
Aeroporto Criado
Aeroporto Criado
Aeroporto Criado
Aeroporto Criado
Aeroporto Criado
Aeroporto Criado
Aeroporto Criado
Aeroporto Criado
Aeroporto Criado
Aeroporto Criado
Aeroporto Criado
Aeroporto Criado
Aeroporto Criado
Aeroporto Criado
Aeroporto Criado
Aeroporto Criado
Aeroporto Criado
Aeroporto Criado
Aeroporto Criado
Aeroporto Criado
Aeroporto Criado
Aeroporto Criado
Aeroporto Criado
Aeroporto Criado
Aeroporto Criado
Aeroporto Criado
Aeroporto Criado
Aeroporto Criado
Aeroporto Criado
Aeroporto Criado
Aeroporto Criado
Aeroporto Criado
Aeroporto Criado
Aeroporto Criado
Aeroporto Criado
Aeroporto Criado
Aeroporto Criado
Aeroporto Criado
Aeroporto Criado
Aeroporto Criado
Aeroporto Criado
Aeroporto Criado
Aeroporto Criado
Aeroporto Criado
Aeroporto Criado
Aeroporto Criado
Aeroporto Criado
Aeroporto Criado
Aeroporto Criado
Aeroporto Criado
Aeroporto Criado
Aeroporto Cria

## Problemas
Com essa maginitude de arestas é impossível trabalhar em tempo hábil. Precisaremos reduzir!<br>
Para reduzir, vamos realizar uma *redução por peso* (ou seja, pelo preço da passagem) e depois realizar uma redução utilizando o algoritmo de **Prim**.

### Grafo Reduzido

In [97]:
class GrafoReduzidoPeso:
    grafo = None
    aeroportos = None
    vertices = 0
    arestas = 0
    def __init__(self):
        self.grafo = {}
        self.aeroportos = {}
    
    def reduzGrafoPorPeso(self, g:Grafo):
        grafo = g.grafo
        aeroportos = g.aeroportos
        for origem in grafo.keys():

            if(not(origem in self.grafo.keys())):
                self.grafo[origem] = {}
                self.vertices +=1
            
            if(not(origem in self.aeroportos.keys())):
                self.aeroportos[origem] = aeroportos[origem]
            
            for destino in grafo[origem].keys():
                lista = [ (value[0], value[1], key) for key, value in grafo[origem][destino].items() ]
                lista.sort()
                self.grafo[origem][destino] = lista[0]
                self.arestas+=1
    
    def adicionarAresta(self, origem, nomeOrigem, destino, nomeDestino, rota):
        #existe origem?
        if(origem in self.grafo.keys()):
        #sim
            self.grafo[origem][destino] = rota
        #nao
        else:
            #adicionar [origem] = {"destino" : {"label" : peso}}
            self.grafo[origem] = {destino : rota}
            self.aeroportos[origem] = nomeOrigem
        
        if(not(destino in self.grafo.keys())):
            self.grafo[destino] = {}
            self.aeroportos[destino] = nomeDestino
    
    def adicionarVertice(self, origem, nomeOrigem):
        if(not(origem in self.grafo.keys())):
            self.grafo[origem] = {}
            self.aeroportos[origem] = nomeOrigem



In [98]:
grafoReduzidoPeso = GrafoReduzidoPeso()
grafoReduzidoPeso.reduzGrafoPorPeso(grafo)

In [99]:
grafoReduzidoPeso.grafo

{'SBSV - BA': {'SIRI - BA': (170, 3.699, 8),
  'SNCL - BA': (112, 11.301, 31),
  'SBBR - DF': (165, 174.183, 90),
  'SBCF - MG': (155, 78.9, 46),
  'SBCY - MT': (290, 2.83333, 1),
  'SBFI - PR': (336, 3.43333, 1),
  'SBGO - GO': (179, 47.85, 23),
  'SBIL - BA': (61, 0.933333, 1),
  'SBKP - SP': (216, 78.6833, 31),
  'SBMK - MG': (251, 23.9667, 12),
  'SBPS - BA': (95, 16.2, 15),
  'SBRF - PE': (100, 31.35, 23),
  'SBRJ - RJ': (215, 10.6167, 5),
  'SBRP - SP': (216, 8.56667, 4),
  'SBSR - SP': (374, 9.46667, 4),
  'SBUL - MG': (393, 13, 4),
  'SBVC - BA': (83, 1.11667, 1),
  'SBGR - SP': (205, 7.48333, 3),
  'SBCT - PR': (265, 8.41667, 3),
  'SBFL - SC': (301, 14.8667, 5),
  'SBFZ - CE': (138, 5.15, 3),
  'SBGL - RJ': (174, 21.0833, 10),
  'SBJP - PB': (105, 26.9833, 18),
  'SBMG - PR': (346, 5.58333, 2),
  'SBMO - AL': (77, 31.0167, 28),
  'SBPA - RS': (357, 13.7667, 4),
  'SBSG - RN': (134, 31.9, 20),
  'SBSL - MA': (199, 38.45, 18),
  'SBSP - SP': (208, 17.1833, 7),
  'SBVT - ES': (1

In [100]:
print("Vértices: ", grafoReduzidoPeso.vertices)
print("Arestas: ",grafoReduzidoPeso.arestas)

Vértices:  164
Arestas:  1160


In [101]:
grafoReduzidoPeso.aeroportos

{'SBSV - BA': 'SALVADOR',
 'SIRI - BA': 'MARAÚ',
 'SNCL - BA': 'CAIRU',
 'SBAC - CE': 'ARACATI',
 'SBFZ - CE': 'FORTALEZA',
 'SBBE - PA': 'BELÉM',
 'SBMD - PA': 'ALMEIRIM',
 'SBSN - PA': 'SANTARÉM',
 'SNEB - PA': 'PARAGOMINAS',
 'SNSM - PA': 'SALINÓPOLIS',
 'SNVS - PA': 'BREVES',
 'SBBG - RS': 'BAGÉ',
 'SBPA - RS': 'PORTO ALEGRE',
 'SBBW - MT': 'BARRA DO GARÇAS',
 'SBCY - MT': 'VÁRZEA GRANDE',
 'SBCF - MG': 'CONFINS',
 'SBVG - MG': 'VARGINHA',
 'SBZM - MG': 'GOIANÁ',
 'SNPD - MG': 'PATOS DE MINAS',
 'SNTO - MG': 'TEÓFILO OTONI',
 'SNZR - MG': 'PARACATU',
 'SBCP - RJ': 'CAMPOS DOS GOYTACAZES',
 'SBRJ - RJ': 'RIO DE JANEIRO',
 'SBCT - PR': 'SÃO JOSÉ DOS PINHAIS',
 'SBJD - SP': 'JUNDIAÍ',
 'SSGY - PR': 'GUAÍRA',
 'SSUM - PR': 'UMUARAMA',
 'SSUV - PR': 'UNIÃO DA VITÓRIA',
 'SSVL - PR': 'TELÊMACO BORBA',
 'SBJI - RO': 'JI-PARANÁ',
 'SSOU - MT': 'ARIPUANÃ',
 'SWHP - MT': 'ÁGUA BOA',
 'SBEG - AM': 'MANAUS',
 'SBMY - AM': 'MANICORÉ',
 'SBTF - AM': 'TEFÉ',
 'SWBC - AM': 'BARCELOS',
 'SWBR - AM'

In [102]:
deletarAllRoutesRoutes()

Rotas Padrão Deletadas


In [103]:
criarRotasPeso(grafoReduzidoPeso)

Rota Peso Criada
Rota Peso Criada
Rota Peso Criada
Rota Peso Criada
Rota Peso Criada
Rota Peso Criada
Rota Peso Criada
Rota Peso Criada
Rota Peso Criada
Rota Peso Criada
Rota Peso Criada
Rota Peso Criada
Rota Peso Criada
Rota Peso Criada
Rota Peso Criada
Rota Peso Criada
Rota Peso Criada
Rota Peso Criada
Rota Peso Criada
Rota Peso Criada
Rota Peso Criada
Rota Peso Criada
Rota Peso Criada
Rota Peso Criada
Rota Peso Criada
Rota Peso Criada
Rota Peso Criada
Rota Peso Criada
Rota Peso Criada
Rota Peso Criada
Rota Peso Criada
Rota Peso Criada
Rota Peso Criada
Rota Peso Criada
Rota Peso Criada
Rota Peso Criada
Rota Peso Criada
Rota Peso Criada
Rota Peso Criada
Rota Peso Criada
Rota Peso Criada
Rota Peso Criada
Rota Peso Criada
Rota Peso Criada
Rota Peso Criada
Rota Peso Criada
Rota Peso Criada
Rota Peso Criada
Rota Peso Criada
Rota Peso Criada
Rota Peso Criada
Rota Peso Criada
Rota Peso Criada
Rota Peso Criada
Rota Peso Criada
Rota Peso Criada
Rota Peso Criada
Rota Peso Criada
Rota Peso Cria

### Redução por Prim

Feito a redução, utilizaremos o algortimo de Prim para reduzir ainda mais nossas arestas.

In [104]:
from math import inf
def algoritmoPrim(g:GrafoReduzidoPeso):
    grafo = g.grafo
    aeroportos = g.aeroportos
    novoGrafo = GrafoReduzidoPeso()
    inicial = list(grafo.keys())[0]
    novoGrafo.adicionarVertice(origem=inicial, nomeOrigem=aeroportos[inicial])
    for n in range(len(grafo)):
        #escolher a melhor aresta
        origem, destino = "", ""
        menorAresta = (inf, inf, inf)
        for v in novoGrafo.grafo.keys(): #dentre os vértices do novo grafo
            for dest, aresta in grafo[v].items(): #veja todas as arestas desses vértices
                if (not(dest in novoGrafo.grafo.keys())): #se o vertice de destino, não está nos vértices que recebem
                  if(aresta[0] < menorAresta[0]): #verifica os pesos
                      menorAresta = aresta
                      origem = v
                      destino = dest 
        if(origem != ""):
            novoGrafo.adicionarAresta(origem=origem, nomeOrigem=aeroportos[origem], destino=destino, nomeDestino=aeroportos[destino], rota=menorAresta)
            origemVolta = destino
            destinoVolta = origem
            if(destinoVolta in grafo[origemVolta].keys()):
                rota = grafo[origemVolta][destinoVolta]
            novoGrafo.adicionarAresta(origem=origemVolta, nomeOrigem=aeroportos[origemVolta], destino=destinoVolta, nomeDestino=aeroportos[destinoVolta], rota=rota)
            # origemVolta = destino
            #dentre as arestas do vértice de destino, qual a menor peso para a volta?
            # destinoVolta = ""
            # menorAresta = (inf, inf, inf)
            # for vertice, aresta in grafo[origemVolta].items():
            #     if(aresta[0] < menorAresta[0]):
            #         menorAresta = aresta
            #         destinoVolta = vertice
            # if(destinoVolta != ""):         
            #     novoGrafo.adicionarAresta(origem=origemVolta, nomeOrigem=aeroportos[origemVolta], destino=destinoVolta, nomeDestino=aeroportos[destinoVolta], rota=menorAresta)
            #     recebidos.append(destinoVolta)
    return novoGrafo


In [105]:
novoGrafo = algoritmoPrim(grafoReduzidoPeso)

In [106]:
len(novoGrafo.grafo)

163

In [107]:
novoGrafo.grafo

{'SBSV - BA': {'SBIL - BA': (61, 0.933333, 1),
  'SBAR - SE': (70, 0.883333, 1),
  'SBMO - AL': (77, 31.0167, 28),
  'SBVC - BA': (83, 1.11667, 1),
  'SBPS - BA': (95, 16.2, 15),
  'SBLE - BA': (97, 2.18333, 2),
  'SNCL - BA': (112, 11.301, 31),
  'SIRI - BA': (170, 3.699, 8),
  'SNTF - BA': (246, 12.2, 8)},
 'SBIL - BA': {'SBSV - BA': (54, 0.75, 1)},
 'SBAR - SE': {'SBSV - BA': (71, 1.01667, 1)},
 'SBMO - AL': {'SBSV - BA': (83, 22.5833, 19), 'SBRF - PE': (55, 0.85, 1)},
 'SBRF - PE': {'SBMO - AL': (67, 0.933333, 1),
  'SBJP - PB': (56, 39.9333, 59),
  'SBKG - PB': (73, 40.1667, 52),
  'SBSG - RN': (80, 25.3667, 28),
  'SBUF - BA': (119, 15.0667, 12),
  'SBMS - RN': (123, 39.3333, 29),
  'SBFN - PE': (128, 153.333, 92),
  'SBPL - PE': (151, 38.4, 28),
  'SBFE - BA': (166, 34.65, 20),
  'SWKQ - PI': (215, 8.68333, 4),
  'SNTS - PB': (483, 29.95, 24),
  'SNGN - PE': (756, 5.41667, 6),
  'SNHS - PE': (1021, 19.05, 12),
  'SNRU - PE': (1114, 8.28333, 12),
  'SNAB - PE': (3168, 32, 9)},
 '

In [108]:
deletarAllRoutesPeso()

Rotas Peso Deletadas


In [109]:
criarRotasPrim(novoGrafo)

Rota Prim Criada
Rota Prim Criada
Rota Prim Criada
Rota Prim Criada
Rota Prim Criada
Rota Prim Criada
Rota Prim Criada
Rota Prim Criada
Rota Prim Criada
Rota Prim Criada
Rota Prim Criada
Rota Prim Criada
Rota Prim Criada
Rota Prim Criada
Rota Prim Criada
Rota Prim Criada
Rota Prim Criada
Rota Prim Criada
Rota Prim Criada
Rota Prim Criada
Rota Prim Criada
Rota Prim Criada
Rota Prim Criada
Rota Prim Criada
Rota Prim Criada
Rota Prim Criada
Rota Prim Criada
Rota Prim Criada
Rota Prim Criada
Rota Prim Criada
Rota Prim Criada
Rota Prim Criada
Rota Prim Criada
Rota Prim Criada
Rota Prim Criada
Rota Prim Criada
Rota Prim Criada
Rota Prim Criada
Rota Prim Criada
Rota Prim Criada
Rota Prim Criada
Rota Prim Criada
Rota Prim Criada
Rota Prim Criada
Rota Prim Criada
Rota Prim Criada
Rota Prim Criada
Rota Prim Criada
Rota Prim Criada
Rota Prim Criada
Rota Prim Criada
Rota Prim Criada
Rota Prim Criada
Rota Prim Criada
Rota Prim Criada
Rota Prim Criada
Rota Prim Criada
Rota Prim Criada
Rota Prim Cria

## Ir de um canto a outro - qual o custo?
Busca em profundidade escolhendo o menor custo. <br>
Guardar o caminho feito, custo em reais e horas.

In [110]:
def busca(grafo, origem, destino, custo, horas, caminho:list, visitado):
    
    visitado.append(origem)
    if(origem == destino):
        return (custo, horas, caminho)
    aVisitar = [(value[0], value[1], value[2], key) for key, value in grafo[origem].items()]
    aVisitar.sort()
    for u in aVisitar:
        if(not(u[3] in visitado)):
            custo += u[0]
            horas += u[1]
            caminho.append(u[3])
            achou = busca(grafo, u[3], destino, custo, horas, caminho, visitado)
            if(achou != False):
                return achou
            custo -= u[0]
            horas -= u[1]
            caminho.pop(-1)
    return False
            #
inicial = "SBMO - AL"
caminho = [inicial]
custo = 0
horas = 0
final = "SWCA - AM"
visitado = []
preco,horas, rota = busca(novoGrafo.grafo, inicial, final, custo, horas, caminho, visitado)


In [111]:
rota

['SBMO - AL',
 'SBRF - PE',
 'SBSG - RN',
 'SBFZ - CE',
 'SBSL - MA',
 'SBBE - PA',
 'SBPV - RO',
 'SBEG - AM',
 'SBUY - AM',
 'SWCA - AM']

In [112]:
horas

274.54994000000005

In [113]:
preco

1021

In [114]:
criarRotasBusca(novoGrafo, rota)

Rota Busca Criada
Rota Busca Criada
Rota Busca Criada
Rota Busca Criada
Rota Busca Criada
Rota Busca Criada
Rota Busca Criada
Rota Busca Criada
Rota Busca Criada
